# Assignment 6.2 - More Trees

Please submit your solution of this notebook in the Whiteboard at the corresponding Assignment entry as .ipynb-file and as .pdf. <br><br>
Please do **NOT** rename the file!

#### State both names of your group members here:
[Jane and John Doe]

In [557]:
# Farah Ahmed Atef Abdelhameed Hafez- Mariz Essam Sobhy Ghaly

---

## Grading Info/Details - Assignment 6.2:

The assignment will be graded semi-automatically, which means that your code will be tested against a set of predefined test cases and qualitatively assessed by a human. This will speed up the grading process for us.

* For passing the test scripts:
    - Please make sure to **NOT** alter predefined class or function names, as this would lead to failing of the test scripts.
    - Please do **NOT** rename the files before uploading to the Whiteboard!

* **(RESULT)** tags indicate checkpoints that will be specifically assessed by a human.

* You will pass the assignment if you pass the majority of test cases and we can at least confirm effort regarding the **(RESULT)**-tagged checkpoints per task.

---

## Task 6.2.1 - Classification Trees

* Implement the Classification Tree Class from scratch using only `NumPy`. The splitting criterion should be Gini impurity. **(RESULT)**
* Run your implementation on the `IRIS` classification dataset. **(RESULT)**

In [558]:
import numpy as np
from sklearn.datasets import load_iris
from abc import ABC, abstractmethod


class TreeNode:
    def __init__(self, feature, threshold, left, right):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right


class DecisionTree(ABC):
    """Base class for a decision tree."""
    def __init__(self,depth=6, random_features=False, minsamples=3):
        self.depth = depth
        self.Random_features = random_features
        self.minsamples=minsamples



    def fit(self, X, y):
        """Build the regression tree."""
        self.X=X
        self.y=y
        thresholds=[]
        self.contordiscrete={}
        for i in range(X.shape[1]):
              uniqpersample=np.unique(X[:,i])
              ratiocontordiscrete=len(uniqpersample)/len(X[:,i])
              if ratiocontordiscrete>=0.9:
                self.contordiscrete[i]="Continous"
              else:
                self.contordiscrete[i]="Discrete"
        self.rootnode = self._build_tree(X, y, depth=0)

    def _build_tree(self, X,  y, depth):
        """Recursively build the tree."""
        if depth < self.depth and len(X)>self.minsamples:
            # print(X.shape[0])

            X_indices=np.arange(X.shape[1])


            Xrandom=X
            if self.Random_features:
              X_indices = np.random.choice(X.shape[1], size=int(np.sqrt(X.shape[1])), replace=False)
              Xrandom= X[:, X_indices]



            thresholds=[]
            for i in range(Xrandom.shape[1]):
              sorted_idx = np.argsort(Xrandom[:, i])
              arr_sorted = Xrandom[sorted_idx, i]
              if self.contordiscrete[X_indices[i]]=="Continous":
                thresholds.append([(arr_sorted[j] + arr_sorted[j+1]) / 2 for j in range(len(arr_sorted) - 1)])
              else:
                thresholds.append([j for j in np.unique(arr_sorted)])

            best_feature, best_threshold = self._find_best_split(Xrandom, y, X_indices, thresholds)
            if best_feature is None or best_threshold is None:
                return self._leaf_value(y)
            X_left, y_left, X_right, y_right=self.splitmydata(X,y, best_feature, best_threshold)
            depth+=1
            # print("left",X.shape[0], X_left.shape[0])
            left = self._build_tree(X_left,y_left,depth)
            # print("right",X.shape[0], X_right.shape[0])
            right = self._build_tree(X_right,y_right, depth)
            return TreeNode(best_feature,best_threshold,left,right)
        return self._leaf_value(y)

    def splitmydata(self, X,y, best_feature, best_threshold):

      Xi = X[:, best_feature]
      left_mask = Xi <= best_threshold
      right_mask = Xi > best_threshold
      y_left=y[left_mask]
      y_right=y[right_mask]
      X_left=X[left_mask]
      X_right=X[right_mask]
      return X_left, y_left, X_right, y_right


    def predict(self, X):
        """Make predictions for X."""
        y_pred=[]
        for x in X:
          res=self.rootnode

          while isinstance(res, TreeNode):
            if x[res.feature]<=res.threshold:
              res=res.left
            else:
              res=res.right
          y_pred.append(res)
        return y_pred

    def _find_best_split(self, X, y,X_indices, thresholds=None):
        """Find the best split."""

        featuremsepair={}
        for i in range(X.shape[1]):
          scorelist=[]
          thresholdlist=[]
          for threshold in thresholds[i]:
            Xi = X[:, i]
            left_mask = Xi <= threshold
            right_mask = Xi > threshold
            y_left=y[left_mask]
            y_right=y[right_mask]
            X_left=X[left_mask]
            X_right=X[right_mask]
            if y_left.size <self.minsamples or y_right.size <self.minsamples:
                continue
            thresholdscore= self._calculate_featurescore(y_left, y_right, y)

            scorelist.append(thresholdscore)
            thresholdlist.append(threshold)
          if len(scorelist)==0:
            continue
          bestthresholdforthisfeature= self.find_best_threshold( scorelist)
          featuremsepair[X_indices[i]]={"bestscore":scorelist[bestthresholdforthisfeature], "threshold":thresholdlist[bestthresholdforthisfeature]}
        if len(featuremsepair)==0:
          return None, None

        best_feature= self.find_bestfeature( featuremsepair)

        bestvalue = featuremsepair[best_feature]

        return best_feature, bestvalue["threshold"]

    @abstractmethod
    def _leaf_value(self, y):
      """Compute the leaf value."""
      pass

    @abstractmethod
    def _calculate_featurescore(self, y_left, y_right):
      pass

    @abstractmethod
    def find_bestfeature(self):
      pass







In [559]:
class ClassificationTree(DecisionTree):
    """Classification decision tree using Gini impurity."""
    def __init__(self, depth=6, minsamples=3,random_features=False):
        super().__init__(depth, random_features, minsamples)


    def _leaf_value(self, y):
        # print(y)
        y_vals, y_counts= np.unique(y, return_counts=True)
        # print(y_vals[np.argmax(y_counts)])
        return y_vals[np.argmax(y_counts)]

    def _calculate_featurescore(self, y_left, y_right, y):

      y_vals, y_counts= np.unique(y, return_counts=True)
      pmk_y=[]
      for i in range(len(y_vals)):
          pmk_y.append(y_counts[i]/len(y))
      temp_y= [pmk*(1-pmk) for pmk in pmk_y]
      gini_index_parent = sum(temp_y)

      y_left_values, y_left_count= np.unique(y_left, return_counts=True)
      pmk_left=[]
      for i in range(len(y_left_values)):
        pmk_left.append(y_left_count[i]/len(y_left))

      y_right_values, y_right_count= np.unique(y_right, return_counts=True)
      pmk_right=[]
      for i in range(len(y_right_values)):
        pmk_right.append(y_right_count[i]/len(y_right))
      temp_left= [pmk*(1-pmk) for pmk in pmk_left]
      gini_index_left = sum(temp_left)
      temp_right= [pmk*(1-pmk) for pmk in pmk_right]
      gini_index_right = sum(temp_right)
      weighted_gini_index = (len(y_left)/(len(y_left)+len(y_right)))*gini_index_left + (len(y_right)/(len(y_left)+len(y_right)))*gini_index_right
      return gini_index_parent-weighted_gini_index


    def find_best_threshold(self, scorelist):
      return np.argmax(scorelist)

    def find_bestfeature(self, featuremsepair):

      return max(featuremsepair, key=lambda i: featuremsepair[i]["bestscore"])








In [560]:
from sklearn.model_selection import train_test_split
X,y=load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tree=ClassificationTree()
tree.fit(X,y)
y_pred=tree.predict(X_test)
print("Accuracy", np.mean(y_test==y_pred))

Accuracy 1.0


## Task 6.2.2 - Random Forests

* Implement Random Forests using only `NumPy`. **(RESULT)**
* Compare the results between the random forest run of your `ClassificationTree` class on the `IRIS` dataset. **(RESULT)**

In [561]:
class RandomForest:
    """Random Forest Classifier."""

    def __init__(self, bags, n_samples, depth=6, minsamples=3):
        self.bags = bags
        self.samples = n_samples
        self.depth=depth
        self.minsamples=minsamples

    def fit(self, X, y):
      self.dict_bagged_trees={}
      for i in range(self.bags):
        bag_indices = np.random.choice(len(X), size=self.samples, replace=True)
        bag_X = X[bag_indices]
        bag_y = y[bag_indices]
        tree = ClassificationTree(self.depth,self.minsamples,random_features=True)
        tree.fit(bag_X, bag_y)
        self.dict_bagged_trees[i]=tree

    def predict(self, X):
        treepred=[]
        finaly_pred=[]
        for i in range(self.bags):
          tree=self.dict_bagged_trees[i]
          y_pred=tree.predict(X)
          treepred.append(y_pred)
        treepred=np.array(treepred)
        for i in range(treepred.shape[1]):
          temp= treepred[:,i]
          y_vals, y_counts= np.unique(temp, return_counts=True)
          finaly_pred.append(y_vals[np.argmax(y_counts)])
        return np.array(finaly_pred)




In [562]:
tree=RandomForest(20, 60)
tree.fit(X,y)
y_pred=tree.predict(X_test)
print("Accuracy", np.mean(y_test==y_pred))

Accuracy 1.0


We can see that both random forest and single tree have very high same results on the iris dataset. This is because it is a toy data set.

## Task 6.2.3 - Extra Trees

* Implement Extra Trees using only `NumPy`. **(RESULT)**
* Compare the results between the `Random Forest` and an `Extra Trees` ensemble implementation on the `IRIS` dataset. **(RESULT)**

In [563]:
class ExtraTree(ClassificationTree):
    """Extremely Randomized Tree - uses random thresholds instead of optimal ones."""

    def __init__(self, n_thresholds, depth=6, minsamples=3):
        super().__init__(depth,minsamples)
        self.n_thresholds = n_thresholds
        self.depth=depth
        self.minsamples=minsamples
        # self.Random_fearure=False

    def _build_tree(self, X, y, depth, thresholds=None):
        """Recursively build the tree."""
        if depth < self.depth:
            thresholds=[]
            X_indices = np.random.choice(X.shape[1], size=int(np.sqrt(X.shape[1])), replace=False)
            Xrandom=X[:, X_indices]
            for i in X_indices:
              if self.contordiscrete[i]=="Continous":
                thresholds.append(np.random.uniform(min(X[:,i]), max(X[:,i]), size=self.n_thresholds))
              else:
                unique_vals = np.unique(X[:, i])
                num_thresholds = min(self.n_thresholds, len(unique_vals))
                thresholds.append(np.random.choice(X[:,i], size=num_thresholds, replace=False))
            self.Random_fearure=False
            best_feature, best_threshold = self._find_best_split(Xrandom, y, X_indices, thresholds)
            if best_feature is None or best_threshold is None:
                return self._leaf_value(y)
            X_left, y_left, X_right, y_right=self.splitmydata( X,y, best_feature, best_threshold)
            depth+=1
            left = self._build_tree(X_left,y_left,depth)
            right = self._build_tree(X_right,y_right, depth)
            return TreeNode(best_feature,best_threshold,left,right)
        return self._leaf_value(y)



In [564]:
# class ExtraTree(DecisionTree):
#     """Extremely Randomized Tree - uses random thresholds instead of optimal ones."""

#     def __init__(self, n_thresholds, depth=6, minsamples=3):
#         super().__init__(depth, False,minsamples)
#         self.n_thresholds = n_thresholds
#         self.depth=depth
#         self.minsamples=minsamples
#         # self.Random_fearure=False

#     def _build_tree(self, X, y, depth, thresholds=None):
#         """Recursively build the tree."""
#         if depth < self.depth:
#             thresholds=[]
#             X_indices = np.random.choice(X.shape[1], size=int(np.sqrt(X.shape[1])), replace=False)
#             Xrandom=X[:, X_indices]
#             for i in X_indices:
#               if self.contordiscrete[i]=="Continous":
#                 thresholds.append(np.random.uniform(min(X[:,i]), max(X[:,i]), size=self.n_thresholds))
#               else:
#                 unique_vals = np.unique(X[:, i])
#                 num_thresholds = min(self.n_thresholds, len(unique_vals))
#                 thresholds.append(np.random.choice(X[:,i], size=num_thresholds, replace=False))
#             self.Random_fearure=False
#             best_feature, best_threshold = self._find_best_split(Xrandom, y, X_indices, thresholds)
#             if best_feature is None or best_threshold is None:
#                 return self._leaf_value(y)
#             X_left, y_left, X_right, y_right=self.splitmydata( X,y, best_feature, best_threshold)
#             depth+=1
#             left = self._build_tree(X_left,y_left,depth)
#             right = self._build_tree(X_right,y_right, depth)
#             return TreeNode(best_feature,best_threshold,left,right)
#         return self._leaf_value(y)

#     def _leaf_value(self, y):
#         # return np.bincount(y).argmax()
#         # print(y)
#         y_vals, y_counts= np.unique(y, return_counts=True)
#         # print(y_vals[np.argmax(y_counts)])
#         return y_vals[np.argmax(y_counts)]

#     def _calculate_featurescore(self, y_left, y_right,y):
#       y_vals, y_counts= np.unique(y, return_counts=True)
#       pmk_y=[]
#       for i in range(len(y_vals)):
#           pmk_y.append(y_counts[i]/len(y))
#       temp_y= [pmk*(1-pmk) for pmk in pmk_y]
#       gini_index_parent = sum(temp_y)

#       y_left_values, y_left_count= np.unique(y_left, return_counts=True)
#       pmk_left=[]
#       for i in range(len(y_left_values)):
#         pmk_left.append(y_left_count[i]/len(y_left))

#       y_right_values, y_right_count= np.unique(y_right, return_counts=True)
#       pmk_right=[]
#       for i in range(len(y_right_values)):
#         pmk_right.append(y_right_count[i]/len(y_right))
#       temp_left= [pmk*(1-pmk) for pmk in pmk_left]
#       gini_index_left = sum(temp_left)
#       temp_right= [pmk*(1-pmk) for pmk in pmk_right]
#       gini_index_right = sum(temp_right)
#       weighted_gini_index = (len(y_left)/(len(y_left)+len(y_right)))*gini_index_left + (len(y_right)/(len(y_left)+len(y_right)))*gini_index_right
#       return gini_index_parent-weighted_gini_index


#     def find_best_threshold(self, scorelist):
#       return np.argmax(scorelist)

#     def find_bestfeature(self, featuremsepair):

#       return max(featuremsepair, key=lambda i: featuremsepair[i]["bestscore"])





We altered the inheritance to inherit from classification tree since it is doing a classification task. But in case, it is not accepted. The original inheritance is found above(Uncomment and test).

In [565]:
class ExtraTrees:
    """Extremely Randomized Trees ensemble."""
    def __init__(self, n_trees, n_thresholds, n_samples=0, bagging=False, depth=6, minsamples=3):
        self.n_trees = n_trees
        self.samples = n_samples
        self.bagging = bagging
        self.n_thresholds = n_thresholds
        self.depth=depth
        self.minsamples=minsamples

    def fit(self, X, y):
      self.dict_trees={}
      if self.bagging:
        for i in range(self.n_trees):
          bag_indices = np.random.choice(len(X), size=self.samples, replace=True)
          bag_X = X[bag_indices]
          bag_y = y[bag_indices]
          tree = ExtraTree(self.n_thresholds, self.depth, self.minsamples)
          tree.fit(bag_X, bag_y)
          self.dict_trees[i]=tree
      else:
        for i in range(self.n_trees):
          tree = ExtraTree(self.n_thresholds, self.depth, self.minsamples)
          tree.fit(X, y)
          self.dict_trees[i]=tree

    def predict(self, X):
        """Make predictions by averaging all trees."""
        treepred=[]
        finaly_pred=[]
        for i in range(self.n_trees):
          tree=self.dict_trees[i]
          y_pred=tree.predict(X)
          # print(y_pred)
          treepred.append(y_pred)
        treepred=np.array(treepred)

        for i in range(treepred.shape[1]):
          temp= treepred[:,i]
          y_vals, y_counts= np.unique(temp, return_counts=True)
          finaly_pred.append(y_vals[np.argmax(y_counts)])
        return np.array(finaly_pred)


#Bagging enabled

In [566]:
tree=ExtraTrees(10,30, True)
tree.fit(X,y)
y_pred=tree.predict(X_test)
print("Accuracy", np.mean(y_test==y_pred))

Accuracy 1.0


#Bagging disabled

In [567]:
tree=ExtraTrees(10,30, False)
tree.fit(X,y)
y_pred=tree.predict(X_test)
print("Accuracy", np.mean(y_test==y_pred))

Accuracy 1.0


As we can see that whether bagging is enabled or not, it has very high performance the same as random forest. It i expected to be better in generalisation in other problems than random forest which is not aparent here beacuse iris dataset is a toy dataset as mentioned before.

## Congratz, you made it! :)